In [0]:
import pandas as pd
import numpy as np
import os 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 
import collections
from tqdm import tqdm
 
from tensorflow.keras.layers import Dropout, GRU, Embedding, Input, Conv1D, SpatialDropout1D, BatchNormalization, Dense, TimeDistributed, LeakyReLU, Dot, Reshape, Activation, GlobalAveragePooling1D, LSTM, Lambda, Concatenate, MaxPooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences,skipgrams, make_sampling_table
from tensorflow.keras.models import Model

from tensorflow.keras.optimizers import SGD, RMSprop , Adam

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
X = np.load('/content/drive/My Drive/nlp/data/X_train.npy', allow_pickle=True)
y = np.load('/content/drive/My Drive/nlp/data/y_train.npy', allow_pickle=True)

X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, test_size=0.4)
X_val, X_test, y_val, y_test = train_test_split(X, y, test_size=0.5)

In [0]:
data_train = []
maxx = 0
for data in X_train:
    if len(data) > maxx:
        maxx = len(data)
print(maxx)

108


In [0]:
def create_index(input_text):
    # TODO#2:edit or replace this function

    # # Create threshold to divide rare word from the chuck of input text
    word_counter = collections.Counter(input_text)
    words = list()
    # THRESH = int(0.001 * len(set(input_text)))
    THRESH = 1
    for word in tqdm(input_text):
        if word_counter[word] >= THRESH:
            words.append(word)

    # words = [word for word in input_text ]
    word_count = list()
    #use set and len to get the number of unique words
    word_count.extend(collections.Counter(words).most_common(len(set(words))))
    #include a token for unknown word
    word_count.append(("UNK",0))
    #print out 10 most frequent words
    print(word_count[:20])
    dictionary = dict()

#     dictionary["for_keras_zero_padding"] = 0
    
    for word in word_count:
        dictionary[word[0]] = len(dictionary)
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    data = list()
    for word in input_text:
        if word not in dictionary:
            data.append(dictionary['UNK'])
        else:
            data.append(dictionary[word])

    

    return data,dictionary, reverse_dictionary

def encode_to_feed(chunk_data, dictionary):
    result = []
    for sentence in chunk_data:
        tmp = []
        for word in sentence:
            if word not in dictionary:
                tmp.append(dictionary['UNK'])
            else:
                tmp.append(dictionary[word])
        result.append(np.array(tmp))
    return np.array(result)


In [0]:
maxlen = 200


# flatten to create a dictionary
all_train_words = set()
for ls in X_train:
    tmp = []
    for data in ls:
        all_train_words.add(data)
        tmp.append(data)
    tmp = np.array(tmp)
    
    
dataset,dictionary, reverse_dictionary=create_index(all_train_words)
    
encoded_X_train = encode_to_feed(X_train, dictionary)
encoded_X_val = encode_to_feed(X_val, dictionary)
encoded_X_test = encode_to_feed(X_test, dictionary)

data_train = pad_sequences(encoded_X_train, maxlen=maxlen, padding='post')
data_val = pad_sequences(encoded_X_val, maxlen=maxlen, padding='post')
data_test = pad_sequences(encoded_X_test, maxlen=maxlen, padding='post')

100%|██████████| 1415/1415 [00:00<00:00, 397517.76it/s]

[('', 1), ('บุพเพสันนิวาสรึเปล่า', 1), ('หมายเลข', 1), ('เสด', 1), ('"\'"', 1), ('ต้องการ', 1), ('ตาม', 1), ('อาทิตย์', 1), ('ต ค', 1), ('พลัส', 1), ('ค่าาาาาาา', 1), ('ต่าง', 1), ('ตี', 1), ('ออมสิน', 1), ('ภาษี', 1), ('แอป', 1), ('จีบ', 1), ('หมุนเวียน', 1), ('ล้าน', 1), ('ที', 1)]


In [0]:
# transform label to my own label [4,5] --> [0, 0, 0, 0, 1, 0]
def transform_label(labels):
  result = []
  for label in labels:
    new_label = np.zeros((maxlen,), dtype=np.int)
    for start, end in label:
      new_label[start:end] = 1
    result.append(new_label)
  return np.array(result)

label_train = transform_label(y_train)
label_val = transform_label(y_val)
label_test = transform_label(y_test)

In [0]:
len(dataset)

1415

In [0]:
dim_embedddings = 100
V = len(dictionary)

#step1: select the embedding of the target word from W
w_inputs = Input(shape=(1, ), dtype='int32')
w = Embedding(V, dim_embedddings)(w_inputs)

#step2: select the embedding of the context word from C
c_inputs = Input(shape=(1, ), dtype='int32')
c  = Embedding(V, dim_embedddings)(c_inputs)

#step3: compute the dot product:c_k*v_j
o = Dot(axes=2)([w, c])
o = Reshape((1,), input_shape=(1, 1))(o)

#step4: normailize dot products into probability
o = Activation('sigmoid')(o)


SkipGram = Model(inputs=[w_inputs, c_inputs], outputs=o)
SkipGram.summary()
opt=Adam(lr=0.01)
SkipGram.compile(loss='binary_crossentropy', optimizer=opt)

for _ in range(300):
    prev_i=0
    #it is likely that your GPU won't be able to handle large input
    #just do it 100000 words at a time
    for i in (range(len(dataset)//1000)):
        #generate skipgrams
        data, labels = skipgrams(sequence=dataset[prev_i*1000:(i*1000)+1000], vocabulary_size=V, window_size=3, negative_samples=6.)
        x = [np.array(x) for x in zip(*data)]
        y = np.array(labels, dtype=np.int32)
        if x:
          loss = SkipGram.train_on_batch(x, y)
        prev_i = i 
        print(loss,i*1000)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 1, 100)       141600      input_1[0][0]                    
_____________________________________________________________________________________________

In [0]:
len(dataset)

1415

In [0]:
def seq2seq_model():

  num_encoder_tokens = len(dictionary)
  num_decoder_tokens = maxlen
  latent_dim = 32

  encoder_inputs = Input(shape=(None,))
  encoder_embedding = Embedding(num_encoder_tokens, latent_dim, weights=[SkipGram.get_weights()[0]])(encoder_inputs)
  x, state_h, state_c = LSTM(latent_dim, return_state=True)(encoder_embedding)
  encoder_states = [state_h, state_c]
  
  decoder_inputs = Input(shape=(None,))
  decoder_embedding = Embedding(num_decoder_tokens, latent_dim)(decoder_inputs)
  decoder_lstm = LSTM(latent_dim, return_sequences=True)(decoder_embedding, initial_state=encoder_states)
  decoder_outputs = Dense(num_decoder_tokens, activation='softmax')(decoder_lstm)

  model = Model(inputs=[encoder_inputs, decoder_inputs], outputs=decoder_outputs)
  model.summary()
  model.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['acc'])
  return model 

In [0]:
def first_model():
  inputs = Input(shape=(maxlen, ))
  x = Embedding(len(dictionary), 100, input_length=maxlen, weights=[SkipGram.get_weights()[0]])(inputs)
  x = SpatialDropout1D(0.3)(x)
  x = Conv1D(100, 5, activation='relu')(x)
  x = MaxPooling1D(5)(x)
  x = BatchNormalization()(x)
  x = Conv1D(150, 3, activation='relu')(x) #200
  x = MaxPooling1D(3)(x)
  x = BatchNormalization()(x)
  x = GlobalAveragePooling1D()(x)
  x = Dense(128, activation='relu')(x) # 300
  x = Dropout(0.3)(x)
  x = Dense(64, activation='relu')(x) # 128
  outputs = Dense(maxlen, activation='sigmoid')(x) 
  model = Model(inputs,outputs)
  model.summary()
  model.compile(optimizer=RMSprop(lr=1e-4), loss='categorical_crossentropy', metrics=['acc']) # 1e-4
  return model

In [0]:
# hyperparameter
epochs = 2000
batch_size = 1024

model = first_model()
model.fit(data_train, label_train, epochs=epochs, validation_data=(data_val, label_val), batch_size=batch_size)

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 200)]             0         
_________________________________________________________________
embedding_8 (Embedding)      (None, 200, 100)          141600    
_________________________________________________________________
spatial_dropout1d_6 (Spatial (None, 200, 100)          0         
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 196, 100)          50100     
_________________________________________________________________
max_pooling1d_12 (MaxPooling (None, 39, 100)           0         
_________________________________________________________________
batch_normalization_12 (Batc (None, 39, 100)           400       
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 37, 150)           4515

In [0]:
RMmodel.fit(data_train, label_train,initial_epoch=1000, epochs=2000, validation_data=(data_val, label_val), batch_size=batch_size)


Train on 5996 samples, validate on 4997 samples
Epoch 1001/2000
5996/5996 [==============================] - 0s 58us/sample - loss: 1.2270 - acc: 0.7625 - val_loss: 1.6826 - val_acc: 0.7212
Epoch 1002/2000
5996/5996 [==============================] - 0s 55us/sample - loss: 1.2215 - acc: 0.7653 - val_loss: 1.7105 - val_acc: 0.7513
Epoch 1003/2000
5996/5996 [==============================] - 0s 56us/sample - loss: 1.2231 - acc: 0.7650 - val_loss: 1.7346 - val_acc: 0.7312
Epoch 1004/2000
5996/5996 [==============================] - 0s 56us/sample - loss: 1.2217 - acc: 0.7675 - val_loss: 1.7152 - val_acc: 0.7252
Epoch 1005/2000
5996/5996 [==============================] - 0s 55us/sample - loss: 1.2209 - acc: 0.7662 - val_loss: 1.6860 - val_acc: 0.7364
Epoch 1006/2000
5996/5996 [==============================] - 0s 55us/sample - loss: 1.2259 - acc: 0.7612 - val_loss: 1.7098 - val_acc: 0.7262
Epoch 1007/2000
5996/5996 [==============================] - 0s 55us/sample - loss: 1.2185 - acc: 0.

In [0]:
label_train.shape